In [1]:
import os
from dotenv import load_dotenv
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_API_KEY=
# OPENAI_API_KEY=
load_dotenv()

True

## simple test


In [2]:
from langchain_core.runnables import (
    RunnablePassthrough, 
    RunnableLambda, 
    RunnableParallel
)

os.environ["LANGCHAIN_PROJECT"] = 'lcel_test'

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

output_parser = StrOutputParser()


# 创建模型实例
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
)

# lcel
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

chain.invoke("ice cream")

"Why did the ice cream truck break down? It just couldn't handle the rocky road!"

In [4]:
prompt.invoke({'topic': 'ice cream'})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})])

In [5]:
llm.invoke(prompt.invoke({'topic': 'ice cream'}))

AIMessage(content="Why did the ice cream truck break down?\nIt couldn't handle the rocky road!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 15, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'chatcmpl-C7feid19hfJF5LLL1SoIpDqMft8j2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b72ca7c5-2002-4adc-a1e5-5c324dc69fdf-0', usage_metadata={'input_tokens': 15, 'output_tokens': 17, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
output_parser.invoke(llm.invoke(prompt.invoke({'topic': 'ice cream'})))

'Why did the ice cream truck break down?\n\nBecause it had too many sundae drivers!'

RunnablePassthrough()

拿到什么，输出就是什么

In [7]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [8]:
chain = RunnablePassthrough() | RunnableLambda(lambda x: x.upper())
chain.invoke("hello")

'HELLO'

## json test

In [9]:
os.environ["LANGCHAIN_PROJECT"] = 'json_test2'

In [10]:
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [11]:
llm = ChatOpenAI(model="gpt-4o")

In [12]:
json_parser = JsonOutputParser()

In [13]:
# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", '''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''),
    ("human", "{input}")
])

# 创建 LCEL 链
chain = (
    {"input": RunnablePassthrough()} 
    | prompt 
    | llm 
    | json_parser
)

In [14]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [15]:
print(prompt[0])
print(prompt[1])

prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}') additional_kwargs={}
prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}') additional_kwargs={}


In [16]:
ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template('''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''), 
     HumanMessagePromptTemplate.from_template("{input}")
    ]
)

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [17]:
result = chain.invoke("John is 20 years old. He is a student at the University of California, Berkeley. He is a very smart student.")

In [18]:
result

{'name': 'John',
 'age': 20,
 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}

## RAG

In [19]:
import os
os.environ["LANGCHAIN_PROJECT"] = 'rag_test'

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [21]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(
    api_key="sk-zk269f115f79bce049181cf14a96b870acf68a9952257e6d",  # 替换为你的实际 API Key
    openai_api_base="https://api.zhizengzeng.com/v1",  # 自定义 base_url
    # 可选参数
    # chunk_size=1,  # 根据代理要求调整
    # check_embedding_ctx_length=False,  # 如果报错超长，可关闭检查
)

embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x144960ac0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1449609a0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://api.zhizengzeng.com/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [22]:
vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=OpenAIEmbeddings()
)

In [24]:
retriever = vectorstore.as_retriever()

In [25]:
retriever.invoke("What do cats like to eat?")

[Document(id='d8a0029c-c5db-4c01-9f1b-d15e8b187583', metadata={}, page_content='Cats love thuna')]

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [27]:
rag_chain.invoke("What do cats like to eat?")

'Tuna'

## tool use


In [28]:
os.environ["LANGCHAIN_PROJECT"] = 'tools_test2'

In [29]:
import numpy as np
from langchain_core.tools import Tool
from langchain_core.tools import tool

In [30]:
@tool
def add(num1: float, num2: float) -> float:
    "Add two numbers."
    return num1 + num2
    
@tool
def subtract(num1: float, num2: float) -> float:
    """
    Subtract two numbers.
    """
    return num1 - num2
    
@tool
def multiply(num1: float, num2: float) -> float:
    """Multiply two float ."""
    return num1 * num2

@tool
def divide(numerator: float, denominator: float) -> float:
    """
    Divides the numerator by the denominator.
    """

    result = numerator / denominator
    return result

@tool
def power(base: float, exponent: float) -> float:
    "Take the base to the exponent power, base^exponent."
    return base**exponent


@tool
def exp(x):
    """
    Calculate the natural exponential $e^x$
    """
    return np.exp(x)

In [31]:
tools = [add, subtract, multiply, divide, power, exp]

In [32]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [34]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

system_template = """
You are a helpful math assistant that uses calculation functions to solve complex math problems step by step.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, tools, prompt)

In [35]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "What is the result of directive of sigmoid(2.5)?"})



> Entering new AgentExecutor chain...

Invoking: `exp` with `{'x': -2.5}`
responded: The derivative of the sigmoid function, often denoted as \(\sigma(x)\), is given by the formula:

\[
\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))
\]

where \(\sigma(x) = \frac{1}{1 + e^{-x}}\).

To find the derivative of the sigmoid function at \(x = 2.5\), we need to:

1. Calculate \(\sigma(2.5)\).
2. Use the derivative formula \(\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))\).

Let's start by calculating \(\sigma(2.5)\):

\[
\sigma(2.5) = \frac{1}{1 + e^{-2.5}}
\]

Now, let's compute \(e^{-2.5}\) and then \(\sigma(2.5)\).

0.0820849986238988
Invoking: `divide` with `{'numerator': 1, 'denominator': 1.0820849986238987}`


0.9241418199787566
Invoking: `subtract` with `{'num1': 1, 'num2': 0.9241418199787566}`
responded: The value of \(\sigma(2.5)\) is approximately \(0.9241\).

Now, let's calculate the derivative \(\sigma'(2.5)\) using the formula:

\[
\sigma'(2.5) = \sigma(2.5) \cdot (1 - \sigma(2.5))


{'input': 'What is the result of directive of sigmoid(2.5)?',
 'output': "The derivative of the sigmoid function at \\(x = 2.5\\), \\(\\sigma'(2.5)\\), is approximately \\(0.0701\\)."}

In [37]:
import torch
import torch.nn.functional as F

In [38]:
F.sigmoid(torch.tensor([2.5])) * (1-F.sigmoid(torch.tensor([2.5])))

tensor([0.0701])